## Speeding up BigQuery queries with BI Engine

To speed up small queries in BigQuery, simply turn on BI Engine.
The Client API remains exactly the same.

Accompanies https://medium.com/@lakshmanok/speeding-up-small-queries-in-bigquery-with-bi-engine-4ac8420a2ef0

In [1]:
from google.cloud import bigquery
from timeit import default_timer as timer
from datetime import timedelta

# Construct a BigQuery client object.
client = bigquery.Client()

def run_query(n=5):
    tot_slotmillis, tot_timeelapsed = 0, timedelta(0)
    for iter in range(n):
        query = """
            SELECT name, state, SUM(number) as total_people
            FROM `bigquery-public-data.usa_names.usa_1910_2013`
            GROUP BY name, state
            ORDER BY total_people DESC
            LIMIT 20
        """
        query_job = client.query(query, bigquery.job.QueryJobConfig(use_query_cache=False))
        df = query_job.result().to_dataframe()
        tot_timeelapsed += (query_job.ended - query_job.started)
        tot_slotmillis += query_job.slot_millis
    print("Job stat: slot_mills={} server_time={}".format(tot_slotmillis/n, tot_timeelapsed/n))

Here, I'm running the query without BI Engine turned on.

In [2]:
run_query()

Job stat: slot_mills=6336.8 server_time=0:00:04.547600


Then, I went to the web console and turned on a 1 GB BI Engine reservation (monthly cost: $30).
Note: It seems to take about 3 minutes for the memory to become available, so this is something
you should consider doing for a few hours at least, not on a per-query basis.

In [17]:
run_query()

Job stat: slot_mills=1610.2 server_time=0:00:01.188300


As you can see, I got a 4x speedup! My code did not change.

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License